Rodrigo Fabrício Meneses - 376176

In [1]:
import pandas as pd
import scipy.stats as stats
import math

# Código Fonte

In [2]:
# Método Congruencial Linear
def mcl(a, b, m, seed, n, uniform=True):
    # uniform: True, tranformará a amostra em uma uniforme [0, 1]
    #          False, retorna mcl normal
    
    x = [seed]
    for i in range(n):
        x.append(((a * x[i] + b) % m))
    x.remove(seed)
    # Tranforma em uniforme [0, 1],
    return [i / m for i in x] if uniform else x

### Suporte para o teste de frequência

In [3]:
def freq_esp(n_nums, n_classes):
     return [n_nums // n_classes] * n_classes

In [4]:
def freq_obs(x, n_classes):
    f_o = [0] * n_classes
    for num in x:
        for i in range(n_classes):
            if num < (i + 1)/n_classes:
                f_o[i] += 1
                break
    return f_o

In [5]:
def qui_quad(f_o, f_e):
    return [(f_o[i] - f_e[i])**2 / f_e[i] for i in range(len(f_e))]

In [30]:
def create_df(f_o, f_e, q_q, n_classes):
    df = pd.DataFrame(index=range(n_classes), \
        columns=['Classes', 'Freq Observada', 'Freq Esperada', 'Qui Quadrado'])
    
    # 10 classes
    if n_classes == 10:
        classes = [f'0.{i}0 - 0.{i + 1}0' for i in range(9)]
        classes.append('0.90 - 1.00')
    
    #100 classes
    if n_classes == 100:
        classes = [f'0.0{i} - 0.0{i + 1}' for i in range(9)]
        classes.append('0.09 - 0.10')
        for i in range(10, 99):
            classes.append(f'0.{i} - 0.{i + 1}')
        classes.append('0.99 - 1.00')
    
    df['Classes'] = classes
    df['Freq Observada'] = f_o
    df['Freq Esperada'] = f_e
    df['Qui Quadrado'] = q_q
    return df

# Testes

### Teste de Frequência

In [9]:
def teste_de_frequencia(x, n_classes, alfa):
    # Tamanho da instancia
    n_nums = len(x)
    # Vetor de Frequências Observadas
    f_o = freq_obs(x, n_classes)
    # Vetor de Frequências Esperadas
    f_e = freq_esp(n_nums, n_classes)
    # Vetor de Qui-Quadrados
    q_q = qui_quad(f_o, f_e)
    
    liberdade = len(f_e) - 1
    
    print('Teste de frequência\n')
    
    print(create_df(f_o, f_e, q_q, n_classes),'\n')

    print(f'H0: O conjunto de dados é uniformemente distribuido')
    print(f'H1: O conjunto de dados NÃO é uniformemente distribuido\n')
    
    print(f'Se o valor de nossa Estatística for menor que o valor tabelado, \
            não rejeitamos nossa hipótese nula, caso contrário, a rejeitamos.\n')

    print(f'{sum(q_q)} : Valor da estatística')
    print(f'{stats.chi2.ppf(1 - alfa, liberdade)} : Valor tabelado da qui-quadrado \
            com {liberdade} graus de liberdade e significância alfa de {alfa}\n')

    if sum(q_q) < stats.chi2.ppf(1 - alfa, liberdade):
        print(f'Com evidência de {(1 - alfa) * 100}%, \
            NÃO REJEITAMOS a hipótese de que os dados são uniformemente distribuidos')
    else:
        print(f'Com evidência de {(1 - alfa) * 100}%, \
            REJEITAMOS a hipótese de que os dados são uniformemente distribuidos')

### Teste de Execução

In [10]:
def teste_de_execucao(x, alfa):
    troca = x[0] > x[1]
    execucoes = 1
    for i in range(1, len(x) - 1):
        if (x[i] > x[i + 1]) != troca:
            troca = not troca
            execucoes += 1

    med_execucoes_esp = (2 * len(x) - 1) / 3
    var_execucoes_esp = (16 * len(x) - 29) / 90

    Z = (execucoes - med_execucoes_esp) / math.sqrt(var_execucoes_esp)

    print('Teste de execução\n')
    
    print(f'Houve {execucoes} execuções\n')

    print(f'H0: Média de execuções esperada = {med_execucoes_esp}')
    print(f'H1: Média de execuções esperada != {med_execucoes_esp}\n')

    print(f'Se o valor de Z estiver em: {stats.norm.ppf(alfa)} < Z < {stats.norm.ppf(1 - alfa)} \
            aceitamos que os números são gerados aleatoriamente, caso contrário, a rejeitamos.\n')

    print(f'{Z} : Valor da estatística\n')

    if stats.norm.ppf(alfa) < Z < stats.norm.ppf(1 - alfa):
        print(f'Com evidência de {(1 - alfa) * 100}%, NÃO REJEITAMOS a hipótese de que os dados são gerados aleatoriamente')
    else:
        print(f'Com evidência de {(1 - alfa) * 100}%, REJEITAMOS a hipótese de que os dados são gerados aleatoriamente')

In [11]:
def executar(x, n_classes=10, alfa=0.05):
    teste_de_frequencia(x, n_classes, alfa)
    teste_de_execucao(x, alfa)

# Amostras PDF e Slides

In [12]:
# Exemplo 1
exemplo_1 = mcl(a=17, b=43, m=100, seed=27, n = 1000)
exemplo_1[:10]

[0.02, 0.77, 0.52, 0.27, 0.02, 0.77, 0.52, 0.27, 0.02, 0.77]

In [13]:
# Exemplo 2, i = 1
exemplo_2_1 = mcl(a=13, b=0, m=2**6, seed=1, n = 1000)
exemplo_2_1[:7]

[0.203125, 0.640625, 0.328125, 0.265625, 0.453125, 0.890625, 0.578125]

In [14]:
# Exemplo 2, i = 2
exemplo_2_2 = mcl(a=13, b=0, m=2**6, seed=2, n = 1000)
exemplo_2_2[:8]

[0.40625, 0.28125, 0.65625, 0.53125, 0.90625, 0.78125, 0.15625, 0.03125]

In [15]:
# Exemplo 2, i = 3
exemplo_2_3 = mcl(a=13, b=0, m=2**6, seed=3, n = 1000)
exemplo_2_3[:7]

[0.609375, 0.921875, 0.984375, 0.796875, 0.359375, 0.671875, 0.734375]

In [16]:
# Exemplo 2, i = 4
exemplo_2_4 = mcl(a=13, b=0, m=2**6, seed=4, n = 1000)
exemplo_2_4[:9]

[0.8125, 0.5625, 0.3125, 0.0625, 0.8125, 0.5625, 0.3125, 0.0625, 0.8125]

In [17]:
# Exemplo 3
exemplo_3 = mcl(a=19, b=0, m=100, seed=63, n = 1000)
exemplo_3[:10]

[0.97, 0.43, 0.17, 0.23, 0.37, 0.03, 0.57, 0.83, 0.77, 0.63]

In [18]:
# Exemplo comercial
exemplo_comercial = mcl(a=7**5, b=0, m=2**31 - 1, seed=123456, n=1000)
exemplo_comercial[:10]

[0.9662122432916482,
 0.12917300273160123,
 0.01065691002209527,
 0.11068674135519506,
 0.31206195676329634,
 0.8253073207220563,
 0.940139375599166,
 0.9224856951844346,
 0.21707896479269442,
 0.4461612708150229]

In [19]:
exemplo_slide_exec = [0.43, 0.32, 0.48, 0.23, 0.90, 0.72, 0.94, 0.11, 0.14, 0.67,
0.61, 0.25, 0.45, 0.56, 0.87, 0.54, 0.01, 0.64, 0.65, 0.32, 0.03,
0.93, 0.08, 0.58, 0.41, 0.32, 0.03, 0.18, 0.90, 0.74, 0.32,
0.75, 0.42, 0.71, 0.66, 0.03, 0.44, 0.99, 0.40, 0.51]

# Experimentos Computacionais

### Dados dos exemplos, n = 1000, 10 classes, alfa = 0.05

In [20]:
print('10 primeiros elementos: ', exemplo_1[:10], '\n')
executar(exemplo_1)

10 primeiros elementos:  [0.02, 0.77, 0.52, 0.27, 0.02, 0.77, 0.52, 0.27, 0.02, 0.77] 

Teste de frequência

       Classes  Freq Observada  Freq Esperada  Qui Quadrado
0  0.00 - 0.10             250            100         225.0
1  0.10 - 0.20               0            100         100.0
2  0.20 - 0.30             250            100         225.0
3  0.30 - 0.40               0            100         100.0
4  0.40 - 0.50               0            100         100.0
5  0.50 - 0.60             250            100         225.0
6  0.60 - 0.70               0            100         100.0
7  0.70 - 0.80             250            100         225.0
8  0.80 - 0.90               0            100         100.0
9  0.90 - 1.00               0            100         100.0 

H0: O conjunto de dados é uniformemente distribuido
H1: O conjunto de dados NÃO é uniformemente distribuido

Se o valor de nossa Estatística for menor que o valor tabelado,             não rejeitamos nossa hipótese nula, caso con

In [21]:
print('10 primeiros elementos: ', exemplo_2_1[:10], '\n')
executar(exemplo_2_1)

10 primeiros elementos:  [0.203125, 0.640625, 0.328125, 0.265625, 0.453125, 0.890625, 0.578125, 0.515625, 0.703125, 0.140625] 

Teste de frequência

       Classes  Freq Observada  Freq Esperada  Qui Quadrado
0  0.00 - 0.10             124            100          5.76
1  0.10 - 0.20              62            100         14.44
2  0.20 - 0.30             126            100          6.76
3  0.30 - 0.40             125            100          6.25
4  0.40 - 0.50              63            100         13.69
5  0.50 - 0.60             126            100          6.76
6  0.60 - 0.70              63            100         13.69
7  0.70 - 0.80             124            100          5.76
8  0.80 - 0.90             125            100          6.25
9  0.90 - 1.00              62            100         14.44 

H0: O conjunto de dados é uniformemente distribuido
H1: O conjunto de dados NÃO é uniformemente distribuido

Se o valor de nossa Estatística for menor que o valor tabelado,             não 

In [22]:
print('10 primeiros elementos: ', exemplo_2_2[:10], '\n')
executar(exemplo_2_2)

10 primeiros elementos:  [0.40625, 0.28125, 0.65625, 0.53125, 0.90625, 0.78125, 0.15625, 0.03125, 0.40625, 0.28125] 

Teste de frequência

       Classes  Freq Observada  Freq Esperada  Qui Quadrado
0  0.00 - 0.10             125            100          6.25
1  0.10 - 0.20             125            100          6.25
2  0.20 - 0.30             125            100          6.25
3  0.30 - 0.40               0            100        100.00
4  0.40 - 0.50             125            100          6.25
5  0.50 - 0.60             125            100          6.25
6  0.60 - 0.70             125            100          6.25
7  0.70 - 0.80             125            100          6.25
8  0.80 - 0.90               0            100        100.00
9  0.90 - 1.00             125            100          6.25 

H0: O conjunto de dados é uniformemente distribuido
H1: O conjunto de dados NÃO é uniformemente distribuido

Se o valor de nossa Estatística for menor que o valor tabelado,             não rejeitamos

In [23]:
print('10 primeiros elementos: ', exemplo_comercial[:10], '\n')
executar(exemplo_comercial)

10 primeiros elementos:  [0.9662122432916482, 0.12917300273160123, 0.01065691002209527, 0.11068674135519506, 0.31206195676329634, 0.8253073207220563, 0.940139375599166, 0.9224856951844346, 0.21707896479269442, 0.4461612708150229] 

Teste de frequência

       Classes  Freq Observada  Freq Esperada  Qui Quadrado
0  0.00 - 0.10             109            100          0.81
1  0.10 - 0.20              98            100          0.04
2  0.20 - 0.30             101            100          0.01
3  0.30 - 0.40             107            100          0.49
4  0.40 - 0.50             102            100          0.04
5  0.50 - 0.60              85            100          2.25
6  0.60 - 0.70             106            100          0.36
7  0.70 - 0.80             100            100          0.00
8  0.80 - 0.90              96            100          0.16
9  0.90 - 1.00              96            100          0.16 

H0: O conjunto de dados é uniformemente distribuido
H1: O conjunto de dados NÃO é uni

In [24]:
print('10 primeiros elementos: ', exemplo_2_4[:10], '\n')
executar(exemplo_2_4)

10 primeiros elementos:  [0.8125, 0.5625, 0.3125, 0.0625, 0.8125, 0.5625, 0.3125, 0.0625, 0.8125, 0.5625] 

Teste de frequência

       Classes  Freq Observada  Freq Esperada  Qui Quadrado
0  0.00 - 0.10             250            100         225.0
1  0.10 - 0.20               0            100         100.0
2  0.20 - 0.30               0            100         100.0
3  0.30 - 0.40             250            100         225.0
4  0.40 - 0.50               0            100         100.0
5  0.50 - 0.60             250            100         225.0
6  0.60 - 0.70               0            100         100.0
7  0.70 - 0.80               0            100         100.0
8  0.80 - 0.90             250            100         225.0
9  0.90 - 1.00               0            100         100.0 

H0: O conjunto de dados é uniformemente distribuido
H1: O conjunto de dados NÃO é uniformemente distribuido

Se o valor de nossa Estatística for menor que o valor tabelado,             não rejeitamos nossa hip

In [25]:
print('10 primeiros elementos: ', exemplo_3[:10], '\n')
executar(exemplo_3)

10 primeiros elementos:  [0.97, 0.43, 0.17, 0.23, 0.37, 0.03, 0.57, 0.83, 0.77, 0.63] 

Teste de frequência

       Classes  Freq Observada  Freq Esperada  Qui Quadrado
0  0.00 - 0.10             100            100           0.0
1  0.10 - 0.20             100            100           0.0
2  0.20 - 0.30             100            100           0.0
3  0.30 - 0.40             100            100           0.0
4  0.40 - 0.50             100            100           0.0
5  0.50 - 0.60             100            100           0.0
6  0.60 - 0.70             100            100           0.0
7  0.70 - 0.80             100            100           0.0
8  0.80 - 0.90             100            100           0.0
9  0.90 - 1.00             100            100           0.0 

H0: O conjunto de dados é uniformemente distribuido
H1: O conjunto de dados NÃO é uniformemente distribuido

Se o valor de nossa Estatística for menor que o valor tabelado,             não rejeitamos nossa hipótese nula, caso con

### Dados do exemplo comercial, já que são mais interessantes

### n = 1000, 10 classes, alfa = 0.05

In [26]:
print('10 primeiros elementos: ', exemplo_comercial[:10], '\n')
executar(exemplo_comercial, n_classes=10)

10 primeiros elementos:  [0.9662122432916482, 0.12917300273160123, 0.01065691002209527, 0.11068674135519506, 0.31206195676329634, 0.8253073207220563, 0.940139375599166, 0.9224856951844346, 0.21707896479269442, 0.4461612708150229] 

Teste de frequência

       Classes  Freq Observada  Freq Esperada  Qui Quadrado
0  0.00 - 0.10             109            100          0.81
1  0.10 - 0.20              98            100          0.04
2  0.20 - 0.30             101            100          0.01
3  0.30 - 0.40             107            100          0.49
4  0.40 - 0.50             102            100          0.04
5  0.50 - 0.60              85            100          2.25
6  0.60 - 0.70             106            100          0.36
7  0.70 - 0.80             100            100          0.00
8  0.80 - 0.90              96            100          0.16
9  0.90 - 1.00              96            100          0.16 

H0: O conjunto de dados é uniformemente distribuido
H1: O conjunto de dados NÃO é uni

### n = 1000, 100 classes, alfa = 0.05

In [27]:
print('10 primeiros elementos: ', exemplo_comercial[:10], '\n')
executar(exemplo_comercial, n_classes=100)

10 primeiros elementos:  [0.9662122432916482, 0.12917300273160123, 0.01065691002209527, 0.11068674135519506, 0.31206195676329634, 0.8253073207220563, 0.940139375599166, 0.9224856951844346, 0.21707896479269442, 0.4461612708150229] 

Teste de frequência

        Classes  Freq Observada  Freq Esperada  Qui Quadrado
0   0.00 - 0.01              12             10           0.4
1   0.01 - 0.02              15             10           2.5
2   0.02 - 0.03              16             10           3.6
3   0.03 - 0.04               9             10           0.1
4   0.04 - 0.05               8             10           0.4
..          ...             ...            ...           ...
95  0.95 - 0.96               9             10           0.1
96  0.96 - 0.97              14             10           1.6
97  0.97 - 0.98               8             10           0.4
98  0.98 - 0.99              12             10           0.4
99  0.99 - 1.00               4             10           3.6

[100 rows x 4 

### n = 100000, 10 classes, alfa = 0.05

In [28]:
exemplo_comercial = mcl(a=7**5, b=0, m=2**31 - 1, seed=123456, n=100000)
print('10 primeiros elementos: ', exemplo_comercial[:10], '\n')
executar(exemplo_comercial, n_classes=10)

10 primeiros elementos:  [0.9662122432916482, 0.12917300273160123, 0.01065691002209527, 0.11068674135519506, 0.31206195676329634, 0.8253073207220563, 0.940139375599166, 0.9224856951844346, 0.21707896479269442, 0.4461612708150229] 

Teste de frequência

       Classes  Freq Observada  Freq Esperada  Qui Quadrado
0  0.00 - 0.10           10029          10000        0.0841
1  0.10 - 0.20            9934          10000        0.4356
2  0.20 - 0.30           10010          10000        0.0100
3  0.30 - 0.40           10001          10000        0.0001
4  0.40 - 0.50            9951          10000        0.2401
5  0.50 - 0.60           10101          10000        1.0201
6  0.60 - 0.70           10107          10000        1.1449
7  0.70 - 0.80            9894          10000        1.1236
8  0.80 - 0.90            9968          10000        0.1024
9  0.90 - 1.00           10005          10000        0.0025 

H0: O conjunto de dados é uniformemente distribuido
H1: O conjunto de dados NÃO é uni

### n = 100000, 100 classes, alfa = 0.05

In [29]:
print('10 primeiros elementos: ', exemplo_comercial[:10], '\n')
executar(exemplo_comercial, n_classes=100)

10 primeiros elementos:  [0.9662122432916482, 0.12917300273160123, 0.01065691002209527, 0.11068674135519506, 0.31206195676329634, 0.8253073207220563, 0.940139375599166, 0.9224856951844346, 0.21707896479269442, 0.4461612708150229] 

Teste de frequência

        Classes  Freq Observada  Freq Esperada  Qui Quadrado
0   0.00 - 0.01            1014           1000         0.196
1   0.01 - 0.02            1028           1000         0.784
2   0.02 - 0.03            1037           1000         1.369
3   0.03 - 0.04             928           1000         5.184
4   0.04 - 0.05             982           1000         0.324
..          ...             ...            ...           ...
95  0.95 - 0.96             991           1000         0.081
96  0.96 - 0.97             977           1000         0.529
97  0.97 - 0.98            1025           1000         0.625
98  0.98 - 0.99             982           1000         0.324
99  0.99 - 1.00            1027           1000         0.729

[100 rows x 4 